In [1]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import csv

# Paths to folders
original_folder = "path_to_original_images_folder"
segmented_folder = "path_to_segmented_images_folder"
output_csv_path = "output_scores.csv"

# Get the list of image files in both folders (ignoring extensions)
original_images = [os.path.splitext(name)[0] for name in os.listdir(original_folder)]
segmented_images = [os.path.splitext(name)[0] for name in os.listdir(segmented_folder)]

# Initialize lists to store scores
image_names = []
psnr_scores = []
ssim_scores = []

for img_name in original_images:
    if img_name in segmented_images:
        original_path = os.path.join(original_folder, img_name + ".jpg")  # Add the desired image extension
        segmented_path = os.path.join(segmented_folder, img_name + ".png")  # Add the desired image extension

        original_image = cv2.imread(original_path)
        segmented_image = cv2.imread(segmented_path)

        # Calculate PSNR
        mse = np.mean((original_image - segmented_image) ** 2)
        psnr = 20 * np.log10(255.0 / np.sqrt(mse))

        # Calculate SSIM
        original_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
        segmented_gray = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)
        ssim_score = ssim(original_gray, segmented_gray)

        # Store scores and image name
        image_names.append(img_name)
        psnr_scores.append(psnr)
        ssim_scores.append(ssim_score)

# Combine data into a list of tuples
data = list(zip(image_names, psnr_scores, ssim_scores))

# Write data to CSV
with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name", "PSNR", "SSIM"])
    writer.writerows(data)

print("Scores saved to", output_csv_path)
